In [70]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, LongType
from pyspark.sql.functions import col,count, countDistinct, min, max, when, sum, lit, coalesce, array, array_compact, size
from connection_formation import conn_parms
from configparser import ConfigParser
import pandas as pd
import numpy as np
from tabulate import tabulate
import logging
from datetime import datetime, timedelta
import time
import logging
from os import path
import os

In [ ]:
#gets the todays datetime in str format
today = str(datetime.now().replace(microsecond=0)).replace(' ','_').replace(':','_')

In [101]:
logging.info('test')

In [71]:
parser = ConfigParser()
parser.read('../../auth.ini')

['../../auth.ini']

In [ ]:
spark_jars = parser.get('CONFIG', 'spark_jars')

In [72]:
sfOptions = conn_parms(parser,'syst','pdm_syst')

In [73]:
pg_options = pyspark_pg(parser,'uat','pdm_qa_data')

In [74]:
def create_spark_session(SparkSession,spark_jars):
    spark = SparkSession.builder \
        .master("local")\
        .appName("Data Diff") \
        .config('spark.jars',spark_jars)\
        .getOrCreate()
    return spark

In [75]:
def read_data(spark, read_format,options,sql_query):
    return spark.read.format(read_format) \
    .options(**options)\
    .option("query", sql_query) \
    .load()

In [76]:
def schema_validation(src, tgt):
    compare_columns = []
    src_schema = pd.DataFrame(src.dtypes,columns=['NAME','DATA_TYPE'],dtype=(str,str))
    tgt_schema = pd.DataFrame(tgt.dtypes,columns=['NAME','DATA_TYPE'],dtype=(str,str))
    src_schema['NAME'] = src_schema['NAME'].str.lower()
    tgt_schema['NAME'] = tgt_schema['NAME'].str.lower()
    schema_merge = pd.merge(src_schema,tgt_schema,how='outer',on='NAME',suffixes=['_SRC','_TGT'],indicator=True)
    compare_columns = schema_merge.query("(_merge == 'both')")['NAME'].to_list()
    schema_diff = schema_merge.query("(_merge == 'left_only') | (_merge == 'right_only') | (DATA_TYPE_SRC!=DATA_TYPE_TGT)").reset_index(drop=True)
    conditions = [(schema_diff['_merge']=='both'), (schema_diff['_merge']=='left_only'), (schema_diff['_merge']=='right_only')]
    choice = ['Datatype mismatch', 'Column available only in SRC table', 'Column available only in TGT table']
    schema_diff['COMMENT'] = np.select(conditions,choice)
    schema_diff.drop(columns=['_merge'],inplace=True)
    pd.set_option('display.width', 500)
    print('Schema Validation:')
    if len(schema_diff) == 0:
        print('Schemas matches between Source and Target')
    else:
        print('Schema difference between source and target: ')
        print(schema_diff.to_markdown(index=False,tablefmt='psql'))
    print('')
    return compare_columns

In [77]:
def lower_case(input_cols):
    split = input_cols.split(',')
    split_cols = [col.strip().lower() for col in split]
    return split_cols

In [78]:
#Validate Unique Key by group by and having on each dataframe
def validate_unique_keys(df_check, columns_list):
    df_uk = df_check.dropDuplicates().groupBy(columns_list).agg(count('*').alias('total')).filter(col('total')>1)
    dups = df_uk.count()
    if dups > 0:
        print('Unique key has {} duplicates. Please re-assess the columns:'.format(dups))
        print(df_uk.show(truncate=False,n=5))
    else:
        print('Unique key has no duplicates.')

In [79]:
#Record count before and after dropping duplicates
def dups_check(df_check):
    row_count = df_check.count()
    df_distinct_count = df_check.dropDuplicates().count()
    
    if row_count != df_distinct_count:
        print('Table has {} rows and duplicate rows: {}'.format(row_count,row_count-df_distinct_count))
    else:
        print('Table has {} rows no duplicate rows'.format(row_count))

In [80]:
#describe dataframe
#column_name  min_value, max_value, min_date, max_date, null values, not null values, distinct values, count,  varchar check (sample value)
def describe_table(spark,desc):
    schema= StructType([StructField('column_name', StringType(), True),StructField('data_type', StringType(), True),StructField('min', StringType(), True), StructField('max', StringType(), True), StructField('row_count', LongType(), False), StructField('row_unique_count', LongType(), False), StructField('null_count', LongType(), True)])
    df_desc = spark.createDataFrame(data= spark.sparkContext.emptyRDD(),schema=schema)
    for tbl_col in desc.dtypes:
        df = desc.select(lit(tbl_col[0]).alias('column_name'),lit(tbl_col[1]).alias('data_type'),min(tbl_col[0]).cast('string').alias('min'),max(tbl_col[0]).cast('string').alias('max'),count(lit(1)).alias('row_count'),countDistinct(tbl_col[0]).alias('row_unique_count'),sum(when((col(tbl_col[0]).isNull()) | (col(tbl_col[0].strip()) == ''),1).otherwise(0)).alias('null_count'))
        df_desc = df_desc.union(df)
    df_desc = df_desc.withColumn('notnull_count',col('row_count')-col('null_count'))
    return df_desc

In [22]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
handler = logging.FileHandler('test.log')
formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
handler.setFormatter(formatter)
logger.addHandler(handler)

In [68]:
def prCyan(skk): print("\033[00m {}\033[00m" .format(skk))
print("\033[23;31;40m Hello World, ")

 Hello World, 


In [25]:
logger.info(src.show())

+-----------+------------+--------------------+---------+-----------+-----+-----------+------+-------+----+----+------------------+-------------------+--------------------+-----------+-------------+--------------+----------------+--------------------+----------+--------------------+----------+-----------+-------------+-------------------------+-------------------------+
|ADDRESS_GID|  ADDRESS_ID|           ADDRESS_1|ADDRESS_2|       CITY|STATE|POSTAL_CODE|COUNTY|COUNTRY|TEXT|TYPE|           PURPOSE|           LATITUDE|           LONGITUDE|ADDRESS_SID|RECORD_STATUS|EFFECTIVE_DATE|TERMINATION_DATE|   DATE_TIME_CREATED|CREATED_BY|  DATE_TIME_MODIFIED|SOURCE_GID|MODIFIED_BY|ROW_AUDIT_KEY|ROW_AUDIT_INSERT_DATETIME|ROW_AUDIT_UPDATE_DATETIME|
+-----------+------------+--------------------+---------+-----------+-----+-----------+------+-------+----+----+------------------+-------------------+--------------------+-----------+-------------+--------------+----------------+--------------------+---

In [81]:
spark = create_spark_session(SparkSession,spark_jars)

In [82]:
src_sql = "select * from address order by address_gid"
src = read_data(spark, snowflake_reader,sfOptions,src_sql)

In [83]:
tgt_sql = "select *,1 AS ID from address order by address_gid"
tgt = read_data(spark, 'jdbc',pg_options,tgt_sql)

In [84]:
#Schema validation
compare_columns = schema_validation(src,tgt)

Schema Validation:
Schema difference between source and target: 
+---------------+-----------------+-----------------+------------------------------------+
| NAME          | DATA_TYPE_SRC   | DATA_TYPE_TGT   | COMMENT                            |
|---------------+-----------------+-----------------+------------------------------------|
| address_gid   | decimal(38,0)   | int             | Datatype mismatch                  |
| address_sid   | decimal(38,0)   | int             | Datatype mismatch                  |
| id            | nan             | int             | Column available only in TGT table |
| row_audit_key | decimal(38,0)   | int             | Datatype mismatch                  |
| source_gid    | decimal(38,0)   | int             | Datatype mismatch                  |
+---------------+-----------------+-----------------+------------------------------------+



In [17]:
compare_columns

['address_1',
 'address_2',
 'address_gid',
 'address_id',
 'address_sid',
 'city',
 'country',
 'county',
 'created_by',
 'date_time_created',
 'date_time_modified',
 'effective_date',
 'latitude',
 'longitude',
 'modified_by',
 'postal_code',
 'purpose',
 'record_status',
 'row_audit_insert_datetime',
 'row_audit_key',
 'row_audit_update_datetime',
 'source_gid',
 'state',
 'termination_date',
 'text',
 'type']

In [85]:
#The no of columns should match between src and tgt
src_pks = ['address_gid','address_1','postal_code']
tgt_pks = ['address_gid','address_1','postal_code']

In [17]:
validate_unique_keys(tgt, tgt_pks) 

Unique key has no duplicates.


In [18]:
dups_check(tgt)

Table has 565721 rows no duplicate rows


In [21]:
describe_table(spark,src).show(truncate=False,n=5)

+-----------+-------------+-----------------+------------------------+---------+----------------+----------+-------------+
|column_name|data_type    |min              |max                     |row_count|row_unique_count|null_count|notnull_count|
+-----------+-------------+-----------------+------------------------+---------+----------------+----------+-------------+
|ADDRESS_GID|decimal(38,0)|20460            |649462                  |565728   |565728          |0         |565728       |
|ADDRESS_ID |string       |000106           |wals mailing            |565728   |533377          |32247     |533481       |
|ADDRESS_1  |string       |\t269 01 76th Ave|Â 1985 W Happy Valley Rd|565728   |335736          |0         |565728       |
|ADDRESS_2  |string       |                 |Â                       |565728   |28277           |356478    |209250       |
|CITY       |string       |                 |Zuni                    |565728   |13322           |2         |565726       |
+-----------+---

In [86]:
#creates join conditions
#Creates select list
def compare_pre(src_pks,tgt_pks,compare_columns):
    try:
        pk_count = len(src_pks)
        pk_condition = []
        row_ind_src_con = ''
        row_ind_tgt_con = ''
        #pk_columns = []
        select_list = []
        column_compare = ''
        conditions = []
        
        for cnt in range(pk_count):
            if cnt == 0:
                row_ind_src_con = 'df_merge.'+src_pks[cnt]+'_src.isNull()'
                row_ind_tgt_con = 'df_merge.'+tgt_pks[cnt]+'_tgt.isNull()'
            else:
                row_ind_src_con = row_ind_src_con+' & df_merge.'+src_pks[cnt]+'_src.isNull()'
                row_ind_tgt_con = row_ind_tgt_con+' & df_merge.'+tgt_pks[cnt]+'_tgt.isNull()'
            #pk_columns.append(src_pks[cnt]+'_src')
            pk_condition.append('src_suffix.'+src_pks[cnt]+'_src.eqNullSafe(tgt_suffix.'+tgt_pks[cnt]+'_tgt)')
            #select_list.append('coalesce('+src_pks[cnt]+'_src,'+tgt_pks[cnt]+'_tgt) as '+src_pks[cnt])
            select_list.append(src_pks[cnt])
        cnt = 0
        for name in compare_columns:
            if not (name in src_pks):
                select_list.append(name+'_src')
                if cnt ==0:
                    column_compare = column_compare+'when(df_merge.'+name+'_src!=df_merge.'+name+'_tgt, lit(\''+name+'\'))'
                else:
                    column_compare = column_compare+'.when(df_merge.'+name+'_src!=df_merge.'+name+'_tgt, lit(\''+name+'\'))'
                cnt = cnt+1
            if not (name in tgt_pks):
                select_list.append(name+'_tgt')
        return pk_condition,row_ind_src_con,row_ind_tgt_con,select_list,column_compare
    except Exception as diff_pre_error:
        print(diff_pre_error)

In [89]:
#creates join conditions
#Creates select list
def compare_pre(src_pks,tgt_pks,compare_columns):
    pk_count = len(src_pks)
    pk_condition = []
    row_ind_src_con = ''
    row_ind_tgt_con = ''
    #pk_columns = []
    select_list = []
    column_compare = ''
    conditions = []
    
    for cnt in range(pk_count):
        if cnt == 0:
            row_ind_src_con = 'df_merge.'+src_pks[cnt]+'_src.isNull()'
            row_ind_tgt_con = 'df_merge.'+tgt_pks[cnt]+'_tgt.isNull()'
        else:
            row_ind_src_con = row_ind_src_con+' & df_merge.'+src_pks[cnt]+'_src.isNull()'
            row_ind_tgt_con = row_ind_tgt_con+' & df_merge.'+tgt_pks[cnt]+'_tgt.isNull()'
        #pk_columns.append(src_pks[cnt]+'_src')
        #col('address_2_src').eqNullSafe(col('address_2_tgt'))
        #pk_condition.append('col(\''+src_pks[cnt]+'_src\').eqNullSafe(col(\''+tgt_pks[cnt]+'_tgt\'))')
        pk_condition.append('src_suffix.' + src_pks[cnt] + '_src.eqNullSafe(tgt_suffix.' + tgt_pks[cnt] + '_tgt)')
        #select_list.append('coalesce('+src_pks[cnt]+'_src,'+tgt_pks[cnt]+'_tgt) as '+src_pks[cnt])
        select_list.append(src_pks[cnt])
    cnt = 0
    for name in compare_columns:
        if not (name in src_pks):
            select_list.append(name+'_src')
            if cnt ==0:
                column_compare = column_compare+'when(df_merge.'+name+'_src!=df_merge.'+name+'_tgt, lit(\''+name+'\'))'
            else:
                column_compare = column_compare+'.when(df_merge.'+name+'_src!=df_merge.'+name+'_tgt, lit(\''+name+'\'))'
            cnt = cnt+1
        if not (name in tgt_pks):
            select_list.append(name+'_tgt')
    return pk_condition,row_ind_src_con,row_ind_tgt_con,select_list,column_compare

In [90]:
pk_condition,row_ind_src_con,row_ind_tgt_con,select_list,column_compare = compare_pre(src_pks,tgt_pks,compare_columns)
pk_condition

['src_suffix.address_gid_src.eqNullSafe(tgt_suffix.address_gid_tgt)',
 'src_suffix.address_1_src.eqNullSafe(tgt_suffix.address_1_tgt)',
 'src_suffix.postal_code_src.eqNullSafe(tgt_suffix.postal_code_tgt)']

In [97]:
#Join dataframes
def compare(src, tgt, src_pks, tgt_pks, compare_columns):
    try:
        src_suffix = src
        tgt_suffix = tgt
        
        for new_col in src_suffix.columns:
            src_suffix = src_suffix.withColumnRenamed(new_col,new_col.lower()+'_src')
        for new_col in tgt.columns:
            tgt_suffix = tgt_suffix.withColumnRenamed(new_col,new_col.lower()+'_tgt')
        pk_condition,row_ind_src_con,row_ind_tgt_con,select_list,column_compare = compare_pre(src_pks,tgt_pks,compare_columns)    
        df_merge = src_suffix.join(tgt_suffix, [eval(con) for con in pk_condition] , how='outer')
        
        #replace PK columns
        nvl = len(src_pks) 
        for cnt in range(nvl):
            re_name = 'coalesce(df_merge.'+src_pks[cnt]+'_src,df_merge.'+tgt_pks[cnt]+'_tgt)'
            df_merge = df_merge.withColumn(src_pks[cnt],eval(re_name))
        df_merge = df_merge.withColumn('row_ind',when(eval(row_ind_tgt_con),'S').when(eval(row_ind_src_con),'T').otherwise('B'))\
                        .withColumn('compare',array_compact(array(eval(column_compare))))
        #Row Counts
        #print(df_merge.printSchema())
        common_count = df_merge.where("row_ind == 'B'").count()
        common_match_count = df_merge.where((col('row_ind') == 'B') & (size(col('compare')) == lit(0))).count()
        common_mismatch_count = df_merge.where((col('row_ind') == 'B') & (size(col('compare')) > lit(0))).count()
        only_src_cnt = df_merge.where("row_ind == 'S'").count()
        only_tgt_cnt = df_merge.where("row_ind == 'T'").count()
        #Print row counts
        print('Row Counts:')
        print('SOURCE ONLY: {} rows'.format(only_src_cnt))
        print('TARGET ONLY: {} rows'.format(only_tgt_cnt))
        print('COMMON: {} rows'.format(common_count))
        print('COMMON COLUMNS WITH ALL MATCHING: {} rows'.format(common_match_count))
        print('COMMON COLUMNS WITH AT LEAST ONE NOT MATCHING: {} rows'.format(common_mismatch_count))
        
        #Print sample rows
        if only_src_cnt>0:
            print('ROWS ONLY IN SOURCE SAMPLE DATA:')
            df_merge.where("row_ind == 'S'").select(*select_list,'row_ind','compare').show(truncate=False, n=5)
        if only_tgt_cnt>0:
            print('ROWS ONLY IN TARGET SAMPLE DATA:')
            df_merge.where("row_ind == 'T'").select(*select_list,'row_ind','compare').show(truncate=False, n=5)
        if common_mismatch_count>0:
            print('COMMON COLUMNS WITH AT LEAST ONE NOT MATCHING SAMPLE DATA:')
            df_merge.where((col('row_ind') == 'B') & (size(col('compare')) > lit(0))).select(*select_list,'row_ind','compare').show(truncate=False, n=5)
        if common_match_count>0:
            print('COMMON ROWS WITH ALL COLUMNS MATCHING SAMPLE DATA:')
            df_merge.where((col('row_ind') == 'B') & (size(col('compare')) == lit(0))).select(*select_list,'row_ind','compare').show(truncate=False, n=5)
    except Exception as diff_error:
        print(diff_error)

In [98]:
compare(src, tgt, src_pks, tgt_pks, compare_columns)

Row Counts:
COMMON: 565721 rows
COMMON COLUMNS WITH ALL MATCHING: 565721 rows
COMMON COLUMNS WITH AT LEAST ONE NOT MATCHING: 0 rows
SOURCE ONLY: 7 rows
TARGET ONLY: 0 rows
ROWS ONLY IN SOURCE SAMPLE DATA:
+-----------+-----------------+-----------+-------------+-------------+--------------+--------------+---------------+---------------+-----------+--------+-----------+-----------+------------+----------+--------------+--------------+---------------------+---------------------+----------------------+----------------------+------------------+------------------+------------+------------+-------------+-------------+---------------+---------------+-----------+-----------+-----------------+-----------------+-----------------------------+-----------------------------+-----------------+-----------------+-----------------------------+-----------------------------+--------------+--------------+---------+---------+--------------------+--------------------+--------+--------+--------+--------+-----

In [206]:
common_match_count = df_merge.where((col('row_ind') == 'B') & (size(col('compare')) == lit(0))).count()

In [207]:
common_match_count

565721

In [209]:
src_suffix.printSchema()

root
 |-- address_gid_src: decimal(38,0) (nullable = false)
 |-- address_id_src: string (nullable = true)
 |-- address_1_src: string (nullable = false)
 |-- address_2_src: string (nullable = true)
 |-- city_src: string (nullable = false)
 |-- state_src: string (nullable = false)
 |-- postal_code_src: string (nullable = false)
 |-- county_src: string (nullable = true)
 |-- country_src: string (nullable = true)
 |-- text_src: string (nullable = true)
 |-- type_src: string (nullable = true)
 |-- purpose_src: string (nullable = true)
 |-- latitude_src: decimal(19,16) (nullable = true)
 |-- longitude_src: decimal(19,16) (nullable = true)
 |-- address_sid_src: decimal(38,0) (nullable = false)
 |-- record_status_src: string (nullable = false)
 |-- effective_date_src: date (nullable = false)
 |-- termination_date_src: date (nullable = false)
 |-- date_time_created_src: timestamp (nullable = false)
 |-- created_by_src: string (nullable = false)
 |-- date_time_modified_src: timestamp (nullable =

In [165]:
df_merge.where("row_ind == 'S'").count()

7

0

565721

In [185]:
df_merge.select(array_compact(array(eval(column_compare))).alias('compare')).show()

+-------+
|compare|
+-------+
|     []|
|     []|
|     []|
|     []|
|     []|
|     []|
|     []|
|     []|
|     []|
|     []|
|     []|
|     []|
|     []|
|     []|
|     []|
|     []|
|     []|
|     []|
|     []|
|     []|
+-------+
only showing top 20 rows



In [68]:
df_merge.select(array(when(df_merge.address_gid_src!=df_merge.address_gid_tgt,'N'),when(df_merge.address_gid_src==df_merge.address_gid_tgt,'N'))).show()

+-------------------------------------------------------------------------------------------------------------------------------+
|array(CASE WHEN (NOT (address_gid_src = address_gid_tgt)) THEN N END, CASE WHEN (address_gid_src = address_gid_tgt) THEN N END)|
+-------------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                      [null, N]|
|                                                                                                                      [null, N]|
|                                                                                                                      [null, N]|
|                                                                                                                      [null, N]|
|                                                                                         